# Clean ZIMAS / zoning file

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import intake
import utils

In [2]:
catalog = intake.open_catalog("../catalogs/*.yml")

bucket_name = 'city-planning-entitlements'

In [3]:
gdf = gpd.read_file(f'zip+s3://{bucket_name}/gis/source/Parcels_2014.zip')

In [4]:
gdf = gdf[gdf.geometry.notna()]

# Subset and keep only obs in City of LA
gdf = gdf[gdf.SITUSCITY == 'LOS ANGELES CA']
gdf = gdf[['AIN', 'geometry']]

In [5]:
utils.make_zipped_shapefile(gdf, '../gis/raw/la_parcels')

Path name: ../gis/raw/la_parcels
Dirname (1st element of path): ../gis/raw/la_parcels
Shapefile name: la_parcels.shp
Shapefile component parts folder: ../gis/raw/la_parcels/la_parcels.shp


In [ ]:
df = pd.read_csv(f's3://{bucket_name}/gis/source/Assessor_Parcels_Data_2019.csv')

In [ ]:
df.to_parquet(f's3://{bucket_name}/data/source/Assessor_Parcels_Data.parquet')

In [ ]:
df.columns

In [ ]:
# Only keep columns on 
keep = ['AIN', 'AssessorID', 'PropertyLocation', 'GeneralUseType', 'SpecificUseType']
df = df[keep]

In [ ]:
df.to_parquet('../data/Assessor_Parcels_2019.parquet')

In [ ]:
gdf = catalog.zoning.read()

In [ ]:
gdf.crs

In [ ]:
df = df[['ZONE_CMPLT']].drop_duplicates()

In [ ]:
df.to_stata('../data/zoning.dta')

In [ ]:
split = df.ZONE_CMPLT.str.split('-', expand = True)

In [ ]:
split[split[3].notna()].head()

In [ ]:
split[split[4].notna()].head()

In [ ]:
df2 = pd.concat([df, split], axis = 1)

In [ ]:
df2[df2[4].notna()].head()

In [ ]:
df.ZONE_SMRY.value_counts()

In [ ]:
df.ZONE_CLASS.value_counts()

In [ ]:
df.ZONE_CMPLT.value_counts()